In [ ]:
!pip install cv2

In [1]:
import cv2
import numpy as np

In [2]:
net = cv2.dnn.readNet("custom-yolov4-tiny-detector_1000.weights","custom-yolov4-tiny-detector.cfg")
classes = []
with open("classes.names",'r') as f:
    classes = f.read().splitlines()

In [ ]:
net = cv2.dnn.readNet("yolov4_custom_final.weights","yolov4_custom.cfg")
classes = []
with open("classes.names",'r') as f:
    classes = f.read().splitlines()

In [3]:
cap = cv2.VideoCapture('Test-objects/Gray-traffic.mp4')
while True:
    _,img = cap.read()
    height ,width,_ = img.shape
    blob = cv2.dnn.blobFromImage(img,1/255,(416,416),(0,0,0),swapRB=True,crop=False)

    net.setInput(blob)

    op_lr_nms = net.getUnconnectedOutLayersNames()

    lrOutput = net.forward(op_lr_nms)


    boxes =[]
    confidences = []
    class_ids = []

    for output in lrOutput:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
        
            if confidence >0.5:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
            
                w = int(detection[2] * width)
                h = int(detection[3] * height)
            
                x = int(center_x - w/2)
                y = int(center_y - h/2)
            
                boxes.append([x,y,w,h])
                confidences.append((float(confidence)))
                class_ids.append(class_id)
            
    indexes = cv2.dnn.NMSBoxes(boxes,confidences,0.5,0.4)

    font = cv2.FONT_HERSHEY_PLAIN
    colors = np.random.uniform(0,255,size =(len(boxes),3))

    if len(indexes)>0:
        for i in indexes.flatten():
            x,y,w,h = boxes[i]
            label = str(classes[class_ids[i]])
            conf = str(round(confidences[i],2))
            color = colors[i]
            cv2.rectangle(img,(x,y),(x+w,y+h),color,2)
            cv2.putText(img,label+" "+conf,(x,y+20),font,2,(255,255,255),2)
    
    cv2.imshow("Image",img)
    key = cv2.waitKey(1)
    if key == 27:
        break
cap.release()    
cv2.destroyAllWindows()
    
    
        